In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from reinforcement.players import DummyPlayer, ModelPlayer
from reinforcement.sequential_dqn.dqn_v2 import DQNTrainerV2
from modeling.chessformer import ChessFormer
from torch.optim import Adam
import torch as t
from copy import deepcopy

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
BUFFER_SIZE = 1_000_000
RANDOM_ACTION_RATE = 0.0
REVERT_MODELS_EVERY_STEP = 1000
BATCH_SIZE = 32
NB_STEPS_REWARD = 3
EPSILON_SAMPLING = 0.001

In [4]:
model_1 = ChessFormer(6, 3, 512, 2048, 64, 8, 0.2).to('cuda')
model_2 = ChessFormer(6, 3, 512, 2048, 64, 8, 0.2).to('cuda')
#model_1_state_dict = t.load('runs/experiment_4/model_1.pt', map_location='cuda')
#model_1.load_state_dict(model_state_dict)
optimizer_1 = Adam(model_1.parameters(), eps=1e-1 ,lr=1e-3)
optimizer_2 = Adam(model_2.parameters(), eps=1e-1, lr=1e-3)
competitor = DummyPlayer()
#competitor_model = deepcopy(model)
#competitor_model.requires_grad_(False)
#competitor = ModelPlayer(competitor_model, random_action_rate=0.0, model_device='cuda')

In [5]:
trainer = DQNTrainerV2(model_1,
                       model_2,
                       optimizer_1,
                       optimizer_2,
                       BUFFER_SIZE,
                       REVERT_MODELS_EVERY_STEP,
                       competitor,
                       BATCH_SIZE,
                       experiment_name='experiment_7',
                       models_device='cuda',
                       nb_steps_reward=NB_STEPS_REWARD,
                       epsilon_sampling=EPSILON_SAMPLING)

2023-03-20 19:46:29.901540: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-20 19:46:30.615451: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-20 19:46:30.615497: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-20 19:46:30.615501: W 

In [6]:
trainer.train(10000)

  0%|          | 7/10000 [01:52<41:46:57, 15.05s/it]

In [ ]:
t.save(model_1.state_dict(), 'runs/experiment_6/model.pt')
t.save(model_2.state_dict(), 'runs/experiment_6/model.pt')